In [ ]:
import json
import requests
import numpy as np
import os
import shutil
from os import path

In [ ]:
#Get google api key
with open("config.json", "r") as f:
    bk = json.load(f)['big_key_demo']
BASE_URL_DIRECTIONS = 'https://maps.googleapis.com/maps/api/directions/json?'
KEY = '&key=' + bk

In [ ]:
#Supporting functions
def get_total_distance(routes_obj):
    return routes_obj.json()['routes'][0]['legs'][0]['distance']['value']

def get_lat_lng(routes_obj):
    lat_lng = []
    steps = routes_obj.json()['routes'][0]['legs'][0]['steps']
    for step in steps:
        end = step['end_location']
        start = step['start_location']
        lat_lng += [(start['lat'],start['lng']),(end['lat'],end['lng'])]
    return lat_lng

def create_path(points):
    path = 'path='
    for i in points:
        path += str(i[0]) + ',' + str(i[1]) + '|'
    return path[:-1] #remove last '|'

def get_coords(r,points):
    for i in r.json()['snappedPoints']:
        points += [(i['location']['latitude'],i['location']['longitude'])]
    return points

def get_snapped_points(unique_points_interpolated,key,BASE_URL_SNAP = 'https://roads.googleapis.com/v1/snapToRoads?',interpolate = '&interpolate=true'):
    points = []
    k = 0
    coords_list = []
    while k <= len(unique_points_interpolated)-1:
        coords_list += [unique_points_interpolated[k]]
        if (len(coords_list)%100==0) or (k+1==len(unique_points_interpolated)): #When we have 100 points or we reach the end of the list.
            path = create_path(coords_list)
            url = BASE_URL_SNAP + path + interpolate + key
            r = requests.get(url)
            points += get_coords(r,points)
            coords_list = []
        k += 1
    return(points)

def interpolate_coordinates(distance, lat_lng, k, separation_mts = 300):
    unique_points = list(set(lat_lng))
    n = max([1,round((distance/separation_mts)/len(unique_points))])
    unique_points_interpolated = []
    for i in range(len(unique_points)-1):
        unique_points_interpolated += list(map(tuple,np.linspace(unique_points[i],unique_points[i+1],n)))
    unique_points_interpolated = sorted(list(set(unique_points_interpolated)), key = lambda x: x[0])
    if n > 1: #If we have any new points to snap. 
        results = get_snapped_points(unique_points_interpolated,k)
        return results
    else:
        return unique_points_interpolated

def create_image(x,origin,destination):
    for heading in range(0,4):
        lat=x[0]
        lng=x[1]
        heading=str(90*heading)
        query='https://maps.googleapis.com/maps/api/streetview?size=400x400&location=%s,%s&fov=90&heading=%s&pitch=10%s' % (str(lat),str(lng),heading,KEY)
        page=requests.get(query)
        filename='%s-%s-%s-%s-%s.jpg' %(origin,destination,str(lat),str(lng),heading)
        if not path.exists(filename+".txt") or os.path.getsize(filename)<5*10^3:
            f = open(filename,'wb')
            f.write(page.content)    
            f.close()

In [ ]:
#Example of waypoints to force the route to pass through a certain road
#waypoints = 'waypoints=1202+Foothill+Blvd+Calistoga|Oakville,CA|Yountville,CA|Napa+Valley+Marriott+Hotel' 
def download_pictures(origin,destination,folder_name,waypoints=None):
    os.chdir('/data/road-scanner')
    
    #Get interpolated coordinates
    if waypoints == None:
        url = BASE_URL_DIRECTIONS + 'origin=' + origin + '&' + 'destination=' + destination + '&' + KEY
    else:
        url = BASE_URL_DIRECTIONS + 'origin=' + origin + '&' + 'destination=' + destination + '&' + waypoints + KEY
    r = requests.get(url)
    upi = interpolate_coordinates(get_total_distance(r),get_lat_lng(r),KEY)
    #Download pictures
    if os.path.exists(folder_name):
        shutil.rmtree(folder_name)
    os.makedirs(folder_name)    
    os.chdir(folder_name)
    org_dest_string='%s-%s' %(origin,destination)
    
    for i in range(len(upi)):
        create_image(upi[i],origin,destination)

In [ ]:
origins = ["320+Acorn+Ct+Vacaville+CA+95688", 
           "12141+Martha+Ann+Dr+Los+Alamitos+CA+90720", "Salida+California", 
           "1625+W+Lugonia+Ave+Redlands+CA+92374","I-205+Tracy+CA+95377"]
destinations = ["4505+W+Capitol+Ave+West+Sacramento+CA+95691", 
                "Wilshire+Federal+Building+11000+Wilshire+Blvd+Los+Angeles+CA+90024", 
                "Golden+State+Hwy,+Bakersfield,+CA+93307",
               "450+N+Atlantic+Blvd,+Monterey+Park,+CA+91754", "Grapevine+California"]
folder_names = ["vacaville_to_sac", 
               "405_westwood_to_long_beach", "99_salida_to_bakersfield", "10_alhambra_to_riverside",
                'I5_tracy_to_grapevine']
for i in range(len(origins)):
    download_pictures(origins[i],destinations[i],folder_names[i])

In [ ]:
### Features of routes missing, replace spaces for '+'
origins = ['Kings+Beach+California+USA','Sand+City+California+USA','Mendocino+California+USA','Death+Valley+Junction+California+92328+USA','Ocotillo+California+92259+USA','Dwight+D+Eisenhower+Hwy+Oakland+CA+94607+USA','Golden+Gate+Bridge+View+Vista+Point+Sausalito+CA+94965+United+States','51+59+Christmas+Tree+Point+Rd+San+Francisco+CA 94131+USA','Fresno+California+USA']
destinations = ['Camp+Richardson+California+96150+USA','Gorda+California+93920+USA','Bodega+Bay+California+94923+USA','CA+190+California+USA','Borrego+Salton+Sea+Way+Thermal+CA+92274+USA','701+Bayshore+Blvd+San+Francisco+CA+94124+USA','The+Palace+Of+Fine+Arts+3601+Lyon+St+San+Francisco+CA+94123+United+States','148+Marview+Way+San+Francisco+CA+94131+USA','Golden+State+Hwy+Bakersfield+CA+93307+USA']
folder_names = ['Lake_Tahoe','Big_Sur_Coast','Northern_Pacific_coast','Death_Valley','Anzo_Borrego','SF_Skyline','Golden_Gate','Twin_Peaks','shorter_99']
for i in range(len(origins)):
    download_pictures(origins[i],destinations[i],folder_names[i])